In [145]:
import re
workflow_rx = r'^(\w+)\{(.*),(\w+)\}$'
step_rx = r'^(\w+)([<>])(\d+):(\w+)$'
rating_rx = r'^(\w+)=(\d+)$'

def parse(input):
  lines = [l.strip() for l in open(input, 'r').readlines()]
  split = lines.index('')
  workflows = [re.match(workflow_rx, l).groups() for l in lines[:split]]
  workflows = [(w[0], [re.match(step_rx, step).groups() for step in w[1].split(',')], w[2]) for w in workflows]
  workflows = [(w[0], [(s[0], s[1], int(s[2]), s[3]) for s in w[1]], w[2]) for w in workflows]
  workflows = dict([(w[0], w[1] + [w[2]]) for w in workflows])
  ratings = [l[1:-1].split(',') for l in lines[split+1:]]
  ratings = [[re.match(rating_rx, q).groups() for q in r] for r in ratings]
  ratings = [dict([(q[0], int(q[1])) for q in r]) for r in ratings]
  return workflows, ratings

parse('sample')

({'px': [('a', '<', 2006, 'qkq'), ('m', '>', 2090, 'A'), 'rfg'],
  'pv': [('a', '>', 1716, 'R'), 'A'],
  'lnx': [('m', '>', 1548, 'A'), 'A'],
  'rfg': [('s', '<', 537, 'gd'), ('x', '>', 2440, 'R'), 'A'],
  'qs': [('s', '>', 3448, 'A'), 'lnx'],
  'qkq': [('x', '<', 1416, 'A'), 'crn'],
  'crn': [('x', '>', 2662, 'A'), 'R'],
  'in': [('s', '<', 1351, 'px'), 'qqz'],
  'qqz': [('s', '>', 2770, 'qs'), ('m', '<', 1801, 'hdj'), 'R'],
  'gd': [('a', '>', 3333, 'R'), 'R'],
  'hdj': [('m', '>', 838, 'A'), 'pv']},
 [{'x': 787, 'm': 2655, 'a': 1222, 's': 2876},
  {'x': 1679, 'm': 44, 'a': 2067, 's': 496},
  {'x': 2036, 'm': 264, 'a': 79, 's': 2244},
  {'x': 2461, 'm': 1339, 'a': 466, 's': 291},
  {'x': 2127, 'm': 1623, 'a': 2188, 's': 1013}])

In [146]:
def get_next(rating, workflow):
  for step in workflow:
    if not isinstance(step, tuple):
      return step

    value = rating[step[0]]
    if step[1] == '<' and value < step[2]:
      return step[3]
    elif step[1] == '>' and value > step[2]:
      return step[3]

def first_part(input):
  workflows, ratings = input
  result = 0

  for rating in ratings:
    current = 'in'
    while current != 'A' and current != 'R':
      current = get_next(rating, workflows[current])
    
    if current == 'A':
      result += sum(rating.values())
  
  return result

assert first_part(parse('sample')) == 19114
first_part(parse('input'))

402185

In [147]:
from functools import reduce

def split(range, step):
  index = 'xmas'.index(step[0])
  if step[1] == '<':
    a = tuple([(r[0], step[2]) if i == index else r for i, r in enumerate(range)])
    b = tuple([(step[2], r[1]) if i == index else r for i, r in enumerate(range)])
  else:
    a = tuple([(step[2] + 1, r[1]) if i == index else r for i, r in enumerate(range)])
    b = tuple([(r[0], step[2] + 1) if i == index else r for i, r in enumerate(range)])
  return a, b

def second_part(input):
  workflows, _ = input
  queue = [('in', ((1, 4001), (1, 4001), (1, 4001), (1, 4001)))]
  result = 0

  while queue:
    name, range = queue.pop()
    if name == 'A':
      result += reduce(lambda a, b: a * b, map(lambda a: a[1] - a[0], range), 1)
    elif name != 'R':
      workflow = workflows[name]
      leftovers = range
      for step in workflow:
        if not isinstance(step, tuple):
          queue.append((step, leftovers))
        else:
          true, false = split(leftovers, step)
          queue.append((step[3], true))
          leftovers = false

  return result

assert second_part(parse('sample')) == 167409079868000
second_part(parse('input'))

130291480568730